# Project Description 
You and your business partner were recently approached by another local business owner who is interested in purchasing Maven Movies. He has lots of questions for 
you about your business and the rental business in general. His offer seems very generous, so you are going to entertain his questions. Leverage your SQL skills to 
extract and analyze data from various tables in the Maven MOveies database to answer your potential Acquirer's questions. 

In [1]:
import os, sys
import mysql.connector as connector
sys.path.append("..")
from display.display_query import *

# Create Connection and Cursor Object

In [2]:
print("Creating a connection between MySQL and Python")
dbconfig={"user":"root", "password":os.environ["MYSQL_ROOT_PASSWORD"], "port":33061, "host":"localhost", "database": "db_mavenmovies"}
connection=connector.connect(**dbconfig)
print("Connection established between MySQL and Python")

print("Creating cursor object from connection")
cursor = connection.cursor()
print("Cursor object created to communicate with MySQL")

Creating a connection between MySQL and Python
Connection established between MySQL and Python
Creating cursor object from connection
Cursor object created to communicate with MySQL


# Show Tables in the database

In [3]:
show_tables_query = """SHOW TABLES;"""
execute_display_query_results(query=show_tables_query, cursor_object=cursor)

+---------------------------------+
|    Tables_in_db_mavenmovies     |
+---------------------------------+
|            tbl_actor            |
|         tbl_actor_award         |
|           tbl_address           |
|           tbl_advisor           |
|          tbl_category           |
|            tbl_city             |
|           tbl_country           |
|          tbl_customer           |
|            tbl_film             |
|         tbl_film_actor          |
|        tbl_film_category        |
|          tbl_film_text          |
|          tbl_inventory          |
|          tbl_investor           |
|          tbl_language           |
|           tbl_payment           |
|           tbl_rental            |
|            tbl_staff            |
|            tbl_store            |
|         view_actor_info         |
|       view_customer_list        |
|         view_film_list          |
| view_nicer_but_slower_film_list |
|   view_sales_by_film_category   |
|       view_sales_by_store 

# List columns in each table 

In [4]:
query = "SHOW TABLES;"
cursor.execute(query)
tables = cursor.fetchall()
for table in tables:
    tbl = table[0]
    column_query = f"""SHOW COLUMNS FROM {tbl}"""
    print(f"{tbl}:")
    execute_display_query_results(query=column_query, cursor_object=cursor)

tbl_actor:
+-------------+-------------------+------+-----+-------------------+-----------------------------------------------+
|    Field    |       Type        | Null | Key |      Default      |                     Extra                     |
+-------------+-------------------+------+-----+-------------------+-----------------------------------------------+
|  actor_id   | smallint unsigned |  NO  | PRI |       NULL        |                auto_increment                 |
| first_name  |    varchar(45)    |  NO  |     |       NULL        |                                               |
|  last_name  |    varchar(45)    |  NO  | MUL |       NULL        |                                               |
| last_update |     timestamp     |  NO  |     | CURRENT_TIMESTAMP | DEFAULT_GENERATED on update CURRENT_TIMESTAMP |
+-------------+-------------------+------+-----+-------------------+-----------------------------------------------+
4 rows returned in time: (0.003 sec)


tbl_actor_awar

# Question One
I will need a list of all staff members, including their first and last names, email addresses, and the store id where they work

In [5]:
select_query = """SELECT first_name, last_name, email, store_id FROM tbl_staff;"""
execute_display_query_results(query=select_query, cursor_object=cursor)

+------------+-----------+------------------------------+----------+
| first_name | last_name |            email             | store_id |
+------------+-----------+------------------------------+----------+
|    Mike    |  Hillyer  | Mike.Hillyer@sakilastaff.com |    1     |
|    Jon     | Stephens  | Jon.Stephens@sakilastaff.com |    2     |
+------------+-----------+------------------------------+----------+
2 rows returned in time: (0.001 sec)




# Question Two
I will need separate counts of inventory items held at each of your two stores

In [6]:
select_query = """SELECT DISTINCT(store_id),
COUNT(inventory_id) OVER(PARTITION BY store_id) AS CountInventory
FROM tbl_inventory;"""
execute_display_query_results(query=select_query, cursor_object=cursor)

+----------+----------------+
| store_id | CountInventory |
+----------+----------------+
|    1     |      2270      |
|    2     |      2311      |
+----------+----------------+
2 rows returned in time: (0.005 sec)




# Question Three 
Please provide a count of active customers for each store.
Also provide a count of all customers email addresses stored in the database 

In [7]:
select_query = """SELECT DISTINCT store_id, countactive FROM 
(SELECT store_id,
COUNT(active) OVER(PARTITION BY store_id ORDER BY store_id) AS CountActive
FROM tbl_customer WHERE active = 1) as t;"""
execute_display_query_results(query=select_query, cursor_object=cursor)

select_query = """SELECT count(DISTINCT email) AS CountEmail FROM tbl_customer;"""
execute_display_query_results(query=select_query, cursor_object=cursor)

+----------+-------------+
| store_id | countactive |
+----------+-------------+
|    1     |     318     |
|    2     |     266     |
+----------+-------------+
2 rows returned in time: (0.002 sec)


+------------+
| CountEmail |
+------------+
|    599     |
+------------+
1 row returned in time: (0.001 sec)




# Question Four 
Please provide a count of unique film titles in the inventory at each store and then provide a count of the unique categories of the films 

In [8]:
select_query = """SELECT store_id,
COUNT(DISTINCT film_id) AS CountFilmTitle 
FROM tbl_inventory
GROUP BY store_id;"""
execute_display_query_results(query=select_query, cursor_object=cursor)

select_query = """SELECT COUNT(DISTINCT name) AS CountCategory FROM tbl_category;"""
execute_display_query_results(query=select_query, cursor_object=cursor)

+----------+----------------+
| store_id | CountFilmTitle |
+----------+----------------+
|    1     |      759       |
|    2     |      762       |
+----------+----------------+
2 rows returned in time: (0.004 sec)


+---------------+
| CountCategory |
+---------------+
|      16       |
+---------------+
1 row returned in time: (0.001 sec)




# Question Five 
Please provide the replacement cost for the film that is least expensive to replace, the most expensive to replace, and the average of all films in the inventory 

In [9]:
select_query = """SELECT 
MIN(replacement_cost) AS MinReplacementCost, 
MAX(replacement_cost) AS MaxReplacementCost,
ROUND(AVG(replacement_cost), 2) AS AvgReplacementCost 
FROM tbl_film;"""
execute_display_query_results(query=select_query, cursor_object=cursor)

+--------------------+--------------------+--------------------+
| MinReplacementCost | MaxReplacementCost | AvgReplacementCost |
+--------------------+--------------------+--------------------+
|        9.99        |       29.99        |       19.98        |
+--------------------+--------------------+--------------------+
1 row returned in time: (0.003 sec)




# Question Six 
Provide the average payment you process as well as the maximum payment processed

In [10]:
select_query = """SELECT 
ROUND(AVG(amount), 2) AS AvgAmountProcessed,
MAX(amount) AS MaxAmountProcessed
FROM tbl_payment;"""
execute_display_query_results(query=select_query, cursor_object=cursor)

+--------------------+--------------------+
| AvgAmountProcessed | MaxAmountProcessed |
+--------------------+--------------------+
|        4.20        |       11.99        |
+--------------------+--------------------+
1 row returned in time: (0.009 sec)




# Question Seven 
Provide a list of all customer identification values, first_name, last_name with a count of rentals they have made all-time, with the highest volume customers at the top of the list. Limit the result to the first 30 customers

In [11]:
select_query = """SELECT DISTINCT(c.customer_id), t.count, c.first_name, c.last_name FROM 
(SELECT customer_id, COUNT(customer_id) OVER(PARTITION BY customer_id) AS Count FROM tbl_rental) as t
INNER JOIN tbl_customer c ON c.customer_id = t.customer_id
ORDER BY count DESC LIMIT 30;"""
execute_display_query_results(query=select_query, cursor_object=cursor)

+-------------+-------+------------+-------------+
| customer_id | count | first_name |  last_name  |
+-------------+-------+------------+-------------+
|     148     |  46   |  ELEANOR   |    HUNT     |
|     526     |  45   |    KARL    |    SEAL     |
|     144     |  42   |   CLARA    |    SHAW     |
|     236     |  42   |   MARCIA   |    DEAN     |
|     75      |  41   |   TAMMY    |   SANDERS   |
|     469     |  40   |   WESLEY   |    BULL     |
|     197     |  40   |    SUE     |   PETERS    |
|     178     |  39   |   MARION   |   SNYDER    |
|     468     |  39   |    TIM     |    CARY     |
|     137     |  39   |   RHONDA   |   KENNEDY   |
|     410     |  38   |   CURTIS   |    IRBY     |
|      5      |  38   | ELIZABETH  |    BROWN    |
|     459     |  38   |   TOMMY    |   COLLAZO   |
|     295     |  38   |   DAISY    |    BATES    |
|     366     |  37   |  BRANDON   |    HUEY     |
|     198     |  37   |   ELSIE    |   KELLEY    |
|     257     |  37   |   MARSH

# Question Eight 
Provide a list of the managers' names at each store with the full address of each property(street address, district, city and country)

In [12]:
select_query = """SELECT * FROM tbl_store;"""
execute_display_query_results(query=select_query, cursor_object=cursor)

select_query = """SELECT staff_id, first_name, last_name, address_id, store_id FROM tbl_staff;"""
execute_display_query_results(query=select_query, cursor_object=cursor)

select_query = """SELECT * FROM tbl_address LIMIT 5;"""
execute_display_query_results(query=select_query, cursor_object=cursor)

select_query = """SELECT * FROM tbl_city LIMIT 5;"""
execute_display_query_results(query=select_query, cursor_object=cursor)

select_query = """SELECT * FROM tbl_country LIMIT 5;"""
execute_display_query_results(query=select_query, cursor_object=cursor)

select_query = """SELECT first_name, last_name, address, district, city, country
FROM tbl_staff AS sf
INNER JOIN tbl_store AS st ON st.store_id = sf.store_id
INNER JOIN tbl_address AS ad ON ad.address_id = sf.address_id
INNER JOIN tbl_city AS c ON c.city_id = ad.city_id
INNER JOIN tbl_country AS co ON c.country_id = co.country_id;"""
execute_display_query_results(query=select_query, cursor_object=cursor)

+----------+------------------+------------+---------------------+
| store_id | manager_staff_id | address_id |     last_update     |
+----------+------------------+------------+---------------------+
|    1     |        1         |     1      | 2006-02-15 04:57:12 |
|    2     |        2         |     2      | 2006-02-15 04:57:12 |
+----------+------------------+------------+---------------------+
2 rows returned in time: (0.001 sec)


+----------+------------+-----------+------------+----------+
| staff_id | first_name | last_name | address_id | store_id |
+----------+------------+-----------+------------+----------+
|    1     |    Mike    |  Hillyer  |     3      |    1     |
|    2     |    Jon     | Stephens  |     4      |    2     |
+----------+------------+-----------+------------+----------+
2 rows returned in time: (0.0 sec)


+------------+----------------------+----------+----------+---------+-------------+-------------+---------------------+
| address_id |       address  

# Question Nine 

<div>
    <p>
    I would like to get a better understanding of all of the inventory that would come along with the business. Please pull together a list of each inventory item you have stocked, 
    including the store_id number, the inventory_id, the name of the film, the film's rating
    and it's rental rate and replacement cost
    </p>
</div>
 

In [21]:
#Results limited to 30 to avoid very large output
select_query = """SELECT store_id, inventory_id, title, rating, rental_rate, replacement_cost
FROM tbl_film AS f
INNER JOIN tbl_inventory AS i ON i.film_id = f.film_id LIMIT 30;"""
execute_display_query_results(query=select_query, cursor_object=cursor)

+----------+--------------+------------------+--------+-------------+------------------+
| store_id | inventory_id |      title       | rating | rental_rate | replacement_cost |
+----------+--------------+------------------+--------+-------------+------------------+
|    1     |      1       | ACADEMY DINOSAUR |   PG   |    0.99     |      20.99       |
|    1     |      2       | ACADEMY DINOSAUR |   PG   |    0.99     |      20.99       |
|    1     |      3       | ACADEMY DINOSAUR |   PG   |    0.99     |      20.99       |
|    1     |      4       | ACADEMY DINOSAUR |   PG   |    0.99     |      20.99       |
|    1     |      16      | AFFAIR PREJUDICE |   G    |    2.99     |      26.99       |
|    1     |      17      | AFFAIR PREJUDICE |   G    |    2.99     |      26.99       |
|    1     |      18      | AFFAIR PREJUDICE |   G    |    2.99     |      26.99       |
|    1     |      19      | AFFAIR PREJUDICE |   G    |    2.99     |      26.99       |
|    1     |      26 

# Question Ten
From the same list of films you just pulled, please roll that data up and provide a summary level overview of your inventory. I would like to know how many inventory items you have with each rating at each store

In [20]:
select_query = """SELECT DISTINCT(rating), 
COUNT(inventory_id) OVER(PARTITION BY rating) AS CountInventory FROM 
(SELECT store_id, inventory_id, title, rating, rental_rate, replacement_cost
FROM tbl_film AS f
INNER JOIN tbl_inventory AS i ON i.film_id = f.film_id) AS t
ORDER BY CountInventory DESC;"""
execute_display_query_results(query=select_query, cursor_object=cursor)

+--------+----------------+
| rating | CountInventory |
+--------+----------------+
| PG-13  |      1018      |
| NC-17  |      944       |
|   PG   |      924       |
|   R    |      904       |
|   G    |      791       |
+--------+----------------+
5 rows returned in time: (0.006 sec)


